<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Regresja" data-toc-modified-id="Regresja-1">Regresja</a></span><ul class="toc-item"><li><span><a href="#Predykcja-z-domyslnymi-parametrami" data-toc-modified-id="Predykcja-z-domyslnymi-parametrami-1.1">Predykcja z domyslnymi parametrami</a></span></li><li><span><a href="#Zadanie-2." data-toc-modified-id="Zadanie-2.-1.2">Zadanie 2.</a></span></li></ul></li></ul></div>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn import neighbors, datasets
from sklearn.datasets import fetch_california_housing, load_boston
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

Przyjrzyjmy się prostemu zestawowi danych do klasyfikacji. Mamy cztery zmienne niezależne i trzy klasy, do których dane przypadki możemy zaliczyć. Wszystkie dane mają charakter numeryczny

In [ ]:
iris = datasets.load_iris()
print(iris.DESCR)

Przygotujmy dane oraz klasyfikator. Dla ułatwienia wykreślania użyjemy tylko dwóch zmiennych.

Zwróć uwagę na element Pipeline, tworzony przez funkcję make_pipeline. Ten obiekt łączy ze sobą kilka róznych estymatorów w jedną całość, na której można używać metod fit i predict. Taka całość pozwala nam dodać normalizację bezpośrednio do samego modelu i przestać myśleć o pośrednich etapach przetwarzania danych.

In [ ]:
X = iris.data[:, :2]
y = iris.target

In [ ]:
#neighbors.KNeighborsClassifier?

In [ ]:
#KNeighborsRegressor?

In [ ]:
#make_pipeline?

In [ ]:
###------------ budowa klasyfikatora knn ---------------------------------
n_neighbors = 5
knn = neighbors.KNeighborsClassifier(n_neighbors, weights='uniform', algorithm='auto')

##------------- pipeline wraz z normalizacją ---------
knn_pipeline = make_pipeline(MinMaxScaler(), knn)    # nowy koncept
knn_pipeline.fit(X, y)

In [ ]:
knn_pipeline.predict?

Wykreślmy jak każdy punkt przestrzeni zostanie przydzielony przez nasz klasyfikator do jednej lub drugiej klasy. Każdy punkt powinien być przydzielony do tej samej klasy, do której trzy najbliższe punkty.

In [ ]:
###-------------- budowa siatki dla wykresu ------------------
h = .02
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
np.c_[xx.ravel(), yy.ravel()]

In [ ]:
np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

In [ ]:
np.arange(x_min, x_max, h).shape, np.arange(y_min, y_max, h).shape, 280 * 220

In [ ]:
np.c_[xx.ravel(), yy.ravel()].shape

In [ ]:
np.c_[xx.ravel(), yy.ravel()]

In [ ]:
##-----------  predykcja dla całej siatki --------
prediction = knn_pipeline.predict(np.c_[xx.ravel(), yy.ravel()])
prediction = prediction.reshape(xx.shape)
prediction

In [ ]:
plt.figure(figsize=(12, 12))

## ---------------- definicje kolorów (hex) ------
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA','#00AAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00','#004D99'])

###------- kolorujemy przestrzen na podstawie siatki ---------
plt.pcolormesh(xx, yy, prediction, cmap=cmap_light, shading='auto')

##--------- nanosimy punkty -----------------
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, edgecolors='black')
plt.title("3-Class classification (k = {0})".format( n_neighbors))
plt.show()

Jakość klasyfikacji możemy ocenić metodami z pakietu `metrics`. Tutaj wykorzystamy miarę dokładności - zwróć uwagę na użycie `.predict()` do otrzymania predykcji z modelu.

In [ ]:
metrics.accuracy_score(y, knn_pipeline.predict(X)) # ocena na zbiorze treningowym - warto dolozyc zbior testowy

In [ ]:
from mlxtend.plotting import plot_decision_regions

# Plotting Decision Regions
fig = plt.figure(figsize=(10, 8))
fig = plot_decision_regions(X,y,clf=knn_pipeline, legend=2)
plt.xlabel('sepal length [cm]')
plt.ylabel('sepal width [cm]')
plt.title('knn on Iris')
plt.show()

## Regresja

In [3]:
boston = load_boston()
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Przygotujmy ramkę danych dla zadania regresji, wybierzmy wartość docelową

In [ ]:
df = pd.DataFrame(boston['data'], columns=boston['feature_names'])
df['y'] = boston['target']
df.head()

### Predykcja z domyslnymi parametrami

Dla domyślnego klasyfikatora/regresora ilość sąsiadów jest równa k=5.

In [ ]:
knn_reg = KNeighborsRegressor()

knn_reg.fit(df.drop(columns=['y']), df['y'])
preds = knn_reg.predict(df.drop(columns=['y']))

In [ ]:
knn_reg.score(df.drop(columns=['y']), df['y']) #R^2

Błąd średniokwadratowy jest naszą miarą dokładności w przypadku zwykłej regresji. Im mniejszy - tym lepszy.

In [ ]:
np.sqrt(mean_squared_error(df['y'], preds))

Dobierzmy więc optymalną liczbę sąsiadów dla naszego modelu. W tym celu sprawdźmy, jak dla różnych wartości k prezentuje się nasz błąd. Zwrócmy uwagę, że nie wykorzystujemy zbioru treningowego i testowego, więc dla k=1 dokładność będzie perfekcyjna - w praktyce powinniśmy dokonać ewaluacji na osobnym zbiorze.

Wykres na zbiorze treningowym ukazuje nam pewną zależność - dodawanie nowych próbek zwieksza nam błąd na zbiorze treningowym. Oznacza to, że dopasowanie do danych spada wraz ze wzrastającym k - jest tak, gdyż im więcej wartości jest uśrednianych, tym bardziej te wartości odstają od średniej tych wartości.

In [ ]:
scores = []

for k in range(1, 50):
    #Narysuj wykres zależności K od RMSE na zbiorze treningowym
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(df.drop(columns=['y']), df['y'])
    preds = knn.predict(df.drop(columns=['y']))
    scores.append(np.sqrt(mean_squared_error(df['y'], preds)))
    
plt.figure(figsize = (8, 6))
plt.xlabel('K')
plt.ylabel('RMSE')
plt.title('K vs RMSE')
plt.plot(range(1,50), scores)

W przypadku ewaluacji na zbiorze testowym zobaczymy zupełnie inną zależności - istnieje optymalne ustawienie k, dla którego dokładność jest największa (tj. błąd jest najmniejszy)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['y']),
                                                   df['y'],
                                                   test_size=.2,
                                                   random_state=1)

In [ ]:
train_error = []
test_error = []

for k in range(1, 10):
    # powtórz wykres jak wyżej dla zbioru testowego i treningowego
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    preds = knn.predict(X_train)
    train_error.append(np.sqrt(mean_squared_error(y_train, preds)))
    
    preds2 = knn.predict(X_test)
    test_error.append(np.sqrt(mean_squared_error(y_test, preds2)))


In [ ]:
plt.figure(figsize=(8, 6))
plt.title('K vs MSE')
plt.xlabel('K')
plt.ylabel('MSE')
plt.plot(range(1,10), train_error, label='train error')
plt.plot(range(1,10), test_error, label='test error')
plt.legend()

### Zadanie 2. 

1. Wyrzucić outliery ze zbioru X_train zaczniemy od usuwania wartości odstających. Metoda IQR - inter-quartile range - określa różnicę między pierwszym i czwartym kwartylem (czyli tytułowe IQR) i traktuje to jako miarę rozciągłości zbioru danych. Wszystkie dane poza 1.5 * IQR od któregokolwiek z kwartyli są wartościami odstającymi,   
2. Zestandaryzować / minmaxscaler X_train i X_test (standard scaler i minmax),  
3. Porównanie metryk: jakie będą wyniki dla p=1, 2, 10
4. Zestandaryzowane zbiory danych wrzucić do pętli powyżej i zbaczyć jak wyglądają train i test mean squared error